#### ResNet
    ResNet诞生于2015年，当年ImageNet竞赛冠军，Top5错误率为3.57%
    Kaiming He, Xiangyu Zhang, Shaoqing Ren. Deep Residual Learning for Image Recognition. In CPVR,2016.
    
    ResNet提出了层间残差跳连，引入了前方信息，缓解梯度消失，使神经网络层数增加成为可能
    
    模型名称      网络层数
    LeNet        5
    AlexNet       8
    VGG         16/19
    InceptionNet v1 22
    
    使用卷积实现特征提取时，神经网络的层数增加，效果会更好。
    ResNet 引入残差结构最主要的目的是解决网络层数不断加深时导致的梯度消失问题。
    
    Inception块中的“+”是沿深度方向叠加（千层蛋糕层数叠加）
    ResNet块中的“+”是特征图对应元素值相加（矩阵值相加）
    
###### ResNet块结构：（实线和虚线均表示恒等映射）
    ①在堆叠卷积前后，维度相同
    实线表示通道相同，计算方式为 H(x) = F(x) + x；
    
    ②在堆叠卷积前后，维度不同
    虚线表示通道不同，计算方式为 H(x) = F(x) + Wx，其中 W 为1*1卷积操作，目的是调整 x 的维度（通道数）。
    
###### ResNet结构：
    in：32*32
    ↓
    C（核：64*3*3，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓(x)
    
    C（核：64*3*3，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：64*3*3，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + x,(x)
    
    C（核：64*3*3，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：64*3*3，步长：1，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + x,(Wx)
    
    C（核：128*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：128*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + Wx,(x)
    
    C（核：128*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：128*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + x,(Wx)
    
    C（核：256*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：256*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + Wx,(x)
    
    C（核：256*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：256*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + x,(Wx)
    
    C（核：512*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：512*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + x,(Wx)
    
    C（核：512*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    ↓
    C（核：512*3*3，步长：2，填充：same ）
    B（Yes）
    A（relu）
    P（None）
    D（None）
    
    ↓H(x) = F(x) + x
    
    global avgpoling
    ↓
    Dense 10

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras import Model

np.set_printoptions(threshold=np.inf)

cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#把两种不同的堆叠方式封装在ResNetBlock类中
class ResnetBlock(Model):

    def __init__(self, filters, strides=1, residual_path=False):
        super(ResnetBlock, self).__init__()
        self.filters = filters
        self.strides = strides
        self.residual_path = residual_path

        self.c1 = Conv2D(filters, (3, 3), strides=strides, padding='same', use_bias=False)
        self.b1 = BatchNormalization()
        self.a1 = Activation('relu')

        self.c2 = Conv2D(filters, (3, 3), strides=1, padding='same', use_bias=False)
        self.b2 = BatchNormalization()
        
#####################使用1*1卷积核操作#######################
        # residual_path为True时，对输入进行下采样，即用1x1的卷积核做卷积操作，保证x能和F(x)维度相同，顺利相加
        if residual_path:#residual_path=1，堆叠卷积层前后不同
            self.down_c1 = Conv2D(filters, (1, 1), strides=strides, padding='same', use_bias=False)
            self.down_b1 = BatchNormalization()
#############################################################        
        self.a2 = Activation('relu')

    def call(self, inputs):
        residual = inputs  # residual等于输入值本身，即residual=x
        # 将输入通过卷积、BN层、激活层，计算F(x)
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)

        x = self.c2(x)
        y = self.b2(x)
        
#####################使用1*1卷积核操作#######################
        if self.residual_path:
            residual = self.down_c1(inputs)
            residual = self.down_b1(residual)
#############################################################
        out = self.a2(y + residual)  # 最后输出的是两部分的和，即F(x)+x或F(x)+Wx,再过激活函数
        return out


class ResNet18(Model):

    def __init__(self, block_list, initial_filters=64):  # block_list表示每个block有几个卷积层
        super(ResNet18, self).__init__()
        self.num_blocks = len(block_list)  # 共有几个block
        self.block_list = block_list
        self.out_filters = initial_filters
        
        self.c1 = Conv2D(self.out_filters, (3, 3), strides=1, padding='same', use_bias=False)#64个3*3的卷积核，步长1，全零填充
        self.b1 = BatchNormalization()#BN操作
        self.a1 = Activation('relu')#relu激活
        
        self.blocks = tf.keras.models.Sequential()
        # 构建ResNet网络结构
        for block_id in range(len(block_list)):  # 第几个resnet block
            for layer_id in range(block_list[block_id]):  # 第几个卷积层

                if block_id != 0 and layer_id == 0:  # 对除第一个block以外的每个block的输入进行下采样
                    block = ResnetBlock(self.out_filters, strides=2, residual_path=True)#虚线连接H(x) = F(x) + Wx
                else:
                    block = ResnetBlock(self.out_filters, residual_path=False)#实线连接H(x) = F(x) + x
                self.blocks.add(block)  # 将构建好的block加入resnet
            self.out_filters *= 2  # 下一个block的卷积核数是上一个block的2倍
            
        self.p1 = tf.keras.layers.GlobalAveragePooling2D()
        
        self.f1 = tf.keras.layers.Dense(10, activation='softmax', kernel_regularizer=tf.keras.regularizers.l2())

    def call(self, inputs):
        x = self.c1(inputs)
        x = self.b1(x)
        x = self.a1(x)
        x = self.blocks(x)
        x = self.p1(x)
        y = self.f1(x)
        return y


model = ResNet18([2, 2, 2, 2])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = "./checkpoint/ResNet18.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True)

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])
model.summary()

# print(model.trainable_variables)
file = open('./weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

###############################################    show   ###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()
